<a href="https://colab.research.google.com/github/vitaldb/respfailure/blob/main/respfailure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Model for Postoperative Respiratory Failure

## Upgrade XGboost library

In [1]:
# latest version xgboost required
!pip uninstall xgboost -y
!pip install xgboost==1.6.1
!pip install ml_insights

Found existing installation: xgboost 1.7.5
Uninstalling xgboost-1.7.5:
  Successfully uninstalled xgboost-1.7.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 4.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for splinecalib: filename=splinecalib-0.0.4-cp39-cp39-linux_x86_64.whl size=124720 sha256=e7578133619a6bdba1f7f0b9205e57a30361ce58ad4382a385cab68754f8be72
  Stored in directory: /root/.cache/pip/wheels/f5/f3/96/c0ba28c4716bbd318957ea8291f859ced603ce754cf3720d36
Successfully built splinecalib


## Download model and sample files

In [2]:
!wget https://github.com/vitaldb/respfailure/raw/main/model.json
!wget https://github.com/vitaldb/respfailure/raw/main/sample.csv
!wget https://github.com/vitaldb/respfailure/raw/main/splinecalib.pkl

--2023-04-09 09:36:39--  https://github.com/vitaldb/respfailure/raw/main/model.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json [following]
--2023-04-09 09:36:39--  https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190490 (186K) [text/plain]
Saving to: ‘model.json’

model.json          100%[===================>] 186.03K  --.-KB/s    in 0.02s   

2023-04-09 09:36:40 (9.67 MB/s) - ‘model.json’ saved [190490/190490]

--2023-04-09 09:36:40--  https://github.com/vitaldb/respfailure/raw/main/sam

## Sample code for single patient

In [3]:
import xgboost as xgb
import numpy as np
import sys
import pickle
import ml_insights as mli

# sample values
alb = 3.4
cr = 4.16
glu = 112
wbc = 6.59
bmi = 24
ptinr = 0.89
age = 78
andur = 105

model = xgb.Booster()
model.load_model('model.json')
with open('splinecalib.pkl', 'rb') as f:
    calib_loaded = pickle.load(f)
x = np.array([alb, cr, glu, wbc, bmi, ptinr, age, andur]).astype(float)
x[x == 0] = np.nan  # use zero for missing value
x = x[None, ...]
y = model.predict(xgb.DMatrix(x))[0]
print(f'Risk (uncalibrated) = {y * 100:.2f}%')
y = calib_loaded.predict(np.array([y]))[0]
print(f'Risk (calibrated) = {y * 100:.2f}%')

Risk (uncalibrated) = 0.99%
Risk (calibrated) = 0.56%


## Sample code using CSV file for multiple patients

In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd

model = xgb.Booster()
model.load_model('model.json')

df = pd.read_csv('sample.csv')
x = df[['alb', 'cr', 'glu', 'wbc', 'bmi', 'ptinr', 'age', 'andur']].astype(float).values
df['pred'] = calib_loaded.predict(model.predict(xgb.DMatrix(x)))
df.to_csv('prediction.csv')
print(df)

    no  andur  age    wbc  ptinr  glu  alb   bmi    cr      pred
0    1    410   40   4.90   1.04   94  4.4  22.5  0.74  0.000484
1    2    100   30   4.20   1.01   96  4.6  19.1  0.66  0.000253
2    3    130   53   5.47   0.98   97  4.2  22.2  0.65  0.000261
3    4    295   59   5.67   1.07   99  4.6  22.4  0.70  0.000674
4    5    260   58   5.34   1.11  103  4.5  26.6  0.87  0.000927
5    6    510   69   4.55   0.96   84  4.0  23.6  0.73  0.004764
6    7    225   66   4.54   1.12  217  3.5  20.1  0.87  0.010682
7    8    355   25  11.10   1.09  135  4.7  24.8  0.93  0.002144
8    9    305   83   8.00   1.10   83  3.2  23.2  0.57  0.025936
9   10     65   56   4.47   0.92  111  4.4  21.2  0.89  0.000275
10  11    150   53  12.80   1.22  141  2.3  33.0  0.53  0.012829
11  12    250   63   4.59   0.97  252  4.5  24.6  0.85  0.001060
12  13    105   62   5.83   1.01  167  4.1  24.7  0.79  0.000429
13  14    115   56   5.58   1.01   83  3.8  21.8  1.05  0.000479
14  15    120   61   8.64